In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
import math
from typing import List, Tuple




In [ ]:
np.random.seed(42)
random.seed(42)

np.set_printoptions(precision=6)

In [ ]:
def bf_closest_points(points: List[Tuple[float, float]]) -> Tuple[int, int, float]:
    n = len(points)
    min_dist = float('inf')
    idx1, idx2 = -1, -1

    for i in range(n):
        for j in range(i + 1, n):
            dx = points[i][0] - points[j][0]
            dy = points[i][1] - points[j][1]
            dist = math.sqrt(dx*dx + dy*dy)

            if dist < min_dist:
                min_dist = dist
                idx1, idx2 = i, j

    return idx1, idx2, min_dist

In [ ]:
def dc_closest_points(points: List[Tuple[float, float]]) -> Tuple[int, int, float]:
    points_with_idx = [(x, y, i) for i, (x, y) in enumerate(points)]
    points_sorted_x = sorted(points_with_idx, key=lambda p: p[0])

    def closest_pair_rec(Px: List[Tuple]) -> Tuple:
        n = len(Px)

        if n <= 3:
            return bf_small(Px)

        mid = n // 2
        Qx = Px[:mid]
        Rx = Px[mid:]

        mid_x = Px[mid][0]

        left_idx1, left_idx2, left_min = closest_pair_rec(Qx)
        right_idx1, right_idx2, right_min = closest_pair_rec(Rx)

        delta = min(left_min, right_min)

        strip_idx1, strip_idx2, strip_min = closest_split_pair(Px, mid_x, delta)

        if left_min <= right_min and left_min <= strip_min:
            return left_idx1, left_idx2, left_min
        elif right_min <= left_min and right_min <= strip_min:
            return right_idx1, right_idx2, right_min
        else:
            return strip_idx1, strip_idx2, strip_min

    def bf_small(points_small: List[Tuple]) -> Tuple:
        """Brute force for base case (n ≤ 3)"""
        n_small = len(points_small)
        min_dist = float('inf')
        idx1, idx2 = -1, -1

        for i in range(n_small):
            for j in range(i + 1, n_small):
                dx = points_small[i][0] - points_small[j][0]
                dy = points_small[i][1] - points_small[j][1]
                dist = math.sqrt(dx*dx + dy*dy)

                if dist < min_dist:
                    min_dist = dist
                    idx1, idx2 = points_small[i][2], points_small[j][2]

        return idx1, idx2, min_dist

    def closest_split_pair(Px: List[Tuple], mid_x: float, delta: float) -> Tuple:
        strip = [p for p in Px if abs(p[0] - mid_x) < delta]

        strip_sorted_y = sorted(strip, key=lambda p: p[1])

        min_dist = delta
        idx1, idx2 = -1, -1

        for i in range(len(strip_sorted_y)):
            for j in range(i + 1, min(i + 8, len(strip_sorted_y))):
                dx = strip_sorted_y[i][0] - strip_sorted_y[j][0]
                dy = strip_sorted_y[i][1] - strip_sorted_y[j][1]
                dist = math.sqrt(dx*dx + dy*dy)

                if dist < min_dist:
                    min_dist = dist
                    idx1, idx2 = strip_sorted_y[i][2], strip_sorted_y[j][2]

        return idx1, idx2, min_dist

    return closest_pair_rec(points_sorted_x)

In [ ]:
def gen_distinct_points(n: int, coord_range: Tuple[float, float] = (0, 1000000)) -> List[Tuple[float, float]]:
    points_set = set()
    max_attempts = n * 10  # Prevent infinite loop

    while len(points_set) < n and max_attempts > 0:
        x = random.uniform(coord_range[0], coord_range[1])
        y = random.uniform(coord_range[0], coord_range[1])
        points_set.add((x, y))
        max_attempts -= 1

    if len(points_set) < n:
        raise ValueError(f"Could not generate {n} distinct points")

    return list(points_set)

def verify_algorithms(points: List[Tuple[float, float]]) -> bool:
    idx1_bf, idx2_bf, dist_bf = bf_closest_points(points)
    idx1_dc, idx2_dc, dist_dc = dc_closest_points(points)

    return math.isclose(dist_bf, dist_dc, rel_tol=1e-9)

In [ ]:
def run_experiments():
    # Input sizes as specified
    # n_values = [10000, 20000, 30000, 40000, 50000,
    #             60000, 70000, 80000, 90000, 100000]

    # Using alternative smaller sizes
    n_values = [10000, 15000, 20000, 25000, 30000,
                35000, 40000, 45000, 50000, 55000]

    m = 10
    results = {
        'n_values': n_values,
        'alg1_times': [],
        'alg2_times': [],
        'alg1_avg_times': [],
        'alg2_avg_times': []
    }

    for n in n_values:
        print(f"Testing n = {n}")

        alg1_iteration_times = []
        alg2_iteration_times = []

        for iteration in range(m):
            points = gen_distinct_points(n)

            start_time = time.time()
            bf_closest_points(points)
            alg1_time = (time.time() - start_time) * 1000
            alg1_iteration_times.append(alg1_time)

            start_time = time.time()
            dc_closest_points(points)
            alg2_time = (time.time() - start_time) * 1000
            alg2_iteration_times.append(alg2_time)

            print(f"  Iteration {iteration + 1}: ALG1 = {alg1_time:.2f}ms, ALG2 = {alg2_time:.2f}ms")

        alg1_avg = np.mean(alg1_iteration_times)
        alg2_avg = np.mean(alg2_iteration_times)

        results['alg1_times'].append(alg1_iteration_times)
        results['alg2_times'].append(alg2_iteration_times)
        results['alg1_avg_times'].append(alg1_avg)
        results['alg2_avg_times'].append(alg2_avg)

        print(f"  Averages: ALG1 = {alg1_avg:.2f}ms, ALG2 = {alg2_avg:.2f}ms")
        print()

    return results

In [ ]:
def calc_cnst_and_tables(results):
    n_values = results['n_values']
    alg1_avg_times = results['alg1_avg_times']
    alg2_avg_times = results['alg2_avg_times']

    print("ALG1 (Brute Force) Table:")
    print("n\tTheoreticalRT\tEmpiricalRT(ms)\tRatio\t\tPredictedRT")

    ratios_alg1 = []
    for i, n in enumerate(n_values):
        theoretical_rt = n ** 2
        empirical_rt = alg1_avg_times[i]
        ratio = empirical_rt / theoretical_rt
        ratios_alg1.append(ratio)

        print(f"{n}\t{theoretical_rt:.2e}\t{empirical_rt:.2f}\t\t{ratio:.2e}\t\t-")

    c1 = max(ratios_alg1)
    print(f"\nConstant c1 = {c1:.2e}")

    print("\nALG2 (Divide & Conquer) Table:")
    print("n\tTheoreticalRT\tEmpiricalRT(ms)\tRatio\t\tPredictedRT")

    ratios_alg2 = []
    for i, n in enumerate(n_values):
        theoretical_rt = n * math.log2(n)
        empirical_rt = alg2_avg_times[i]
        ratio = empirical_rt / theoretical_rt
        ratios_alg2.append(ratio)

        print(f"{n}\t{theoretical_rt:.2f}\t\t{empirical_rt:.2f}\t\t{ratio:.2e}\t\t-")

    c2 = max(ratios_alg2)
    print(f"\nConstant c2 = {c2:.2e}")

    return c1, c2, ratios_alg1, ratios_alg2

def gen_predictions(n_values, c1, c2):
    pred_alg1 = [c1 * (n ** 2) for n in n_values]
    pred_alg2 = [c2 * n * math.log2(n) for n in n_values]

    return pred_alg1, pred_alg2

In [ ]:
def create_plots(results, pred_alg1, pred_alg2):
    n_values = results['n_values']
    alg1_avg_times = results['alg1_avg_times']
    alg2_avg_times = results['alg2_avg_times']

    plt.style.use('seaborn-v0_8')
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Graph 1: Empirical Running Time Comparison
    axes[0].plot(n_values, alg1_avg_times, 'ro-', label='ALG1 Empirical', linewidth=2, markersize=6)
    axes[0].plot(n_values, alg2_avg_times, 'bo-', label='ALG2 Empirical', linewidth=2, markersize=6)
    axes[0].set_xlabel('Input Size (n)')
    axes[0].set_ylabel('Running Time (ms)')
    axes[0].set_title('Empirical Running Time Comparison')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Graph 2: ALG1 Empirical vs Predicted
    axes[1].plot(n_values, alg1_avg_times, 'ro-', label='ALG1 Empirical', linewidth=2, markersize=6)
    axes[1].plot(n_values, pred_alg1, 'g--', label='ALG1 Predicted', linewidth=2)
    axes[1].set_xlabel('Input Size (n)')
    axes[1].set_ylabel('Running Time (ms)')
    axes[1].set_title('ALG1: Empirical vs Predicted RT')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # Graph 3: ALG2 Empirical vs Predicted
    axes[2].plot(n_values, alg2_avg_times, 'bo-', label='ALG2 Empirical', linewidth=2, markersize=6)
    axes[2].plot(n_values, pred_alg2, 'm--', label='ALG2 Predicted', linewidth=2)
    axes[2].set_xlabel('Input Size (n)')
    axes[2].set_ylabel('Running Time (ms)')
    axes[2].set_title('ALG2: Empirical vs Predicted RT')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('closest_pair_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def main():
    print("=== Closest Pair of Points Project ===\n")

    print("Verifying algorithms on small input...")
    test_points = gen_distinct_points(10, (0, 100))
    if verify_algorithms(test_points):
        print(" --------- Algorithms verified - both return same results")
    else:
        print(" --------- Warning: Algorithms may have discrepancies")

    print("\n Running experiments...")
    results = run_experiments()

    print("\n Calculating constants and creating tables...")
    c1, c2, ratios_alg1, ratios_alg2 = calc_cnst_and_tables(results)

    pred_alg1, pred_alg2 = gen_predictions(results['n_values'], c1, c2)

    print("\n Generating plots...")
    create_plots(results, pred_alg1, pred_alg2)

    print("\n Saving results...")
    np.savez('project_results.npz',
             n_values=results['n_values'],
             alg1_avg_times=results['alg1_avg_times'],
             alg2_avg_times=results['alg2_avg_times'],
             c1=c1, c2=c2)


if __name__ == "__main__":
    main()

=== Closest Pair of Points Project ===

Verifying algorithms on small input...
 --------- Algorithms verified - both return same results

 Running experiments...
Testing n = 10000
  Iteration 1: ALG1 = 13105.01ms, ALG2 = 79.12ms
  Iteration 2: ALG1 = 14891.95ms, ALG2 = 169.89ms
  Iteration 3: ALG1 = 13521.24ms, ALG2 = 85.48ms
  Iteration 4: ALG1 = 14004.34ms, ALG2 = 85.12ms
  Iteration 5: ALG1 = 13763.28ms, ALG2 = 79.48ms
  Iteration 6: ALG1 = 13692.76ms, ALG2 = 79.79ms
  Iteration 7: ALG1 = 14628.74ms, ALG2 = 140.30ms
  Iteration 8: ALG1 = 14773.16ms, ALG2 = 87.22ms
  Iteration 9: ALG1 = 14011.17ms, ALG2 = 80.53ms
  Iteration 10: ALG1 = 14396.52ms, ALG2 = 87.16ms
  Averages: ALG1 = 14078.82ms, ALG2 = 97.41ms

Testing n = 15000
  Iteration 1: ALG1 = 39127.48ms, ALG2 = 130.52ms
  Iteration 2: ALG1 = 38749.32ms, ALG2 = 136.00ms
  Iteration 3: ALG1 = 37607.84ms, ALG2 = 131.01ms
  Iteration 4: ALG1 = 37643.38ms, ALG2 = 228.36ms
  Iteration 5: ALG1 = 40011.40ms, ALG2 = 130.56ms
  Iteration 

In [ ]:
# From your logs:
n_values = np.array([10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000])

alg1_avg_times = np.array([
    14078.82,   # n = 10000
    40020.38,   # n = 15000
    82462.58,   # n = 20000
    146600.37,  # n = 25000
    216849.63,  # n = 30000
    306527.47,  # n = 35000
    448205.52,  # n = 40000
    533238.44,  # n = 45000
])

alg2_avg_times = np.array([
    97.41,
    152.75,
    229.29,
    237.89,
    303.40,
    365.58,
    454.02,
    476.75,
])

np.savez('project_results_partial_backup.npz',
         n_values=n_values,
         alg1_avg_times=alg1_avg_times,
         alg2_avg_times=alg2_avg_times)

print("Manually reconstructed data saved as 'project_results_partial_backup.npz'")


Manually reconstructed data saved as 'project_results_partial_backup.npz'


In [ ]:
def run_experiments_again():
    n_values_remaining = [50000, 55000]

    m = 10
    results_remaining = {
        'n_values': [],
        'alg1_avg_times': [],
        'alg2_avg_times': []
    }

    for n in n_values_remaining:
        print(f"Testing n = {n}")

        alg1_iteration_times = []
        alg2_iteration_times = []

        for iteration in range(m):
            points = gen_distinct_points(n)

            start_time = time.time()
            bf_closest_points(points)
            alg1_time = (time.time() - start_time) * 1000
            alg1_iteration_times.append(alg1_time)

            start_time = time.time()
            dc_closest_points(points)
            alg2_time = (time.time() - start_time) * 1000
            alg2_iteration_times.append(alg2_time)

            print(f"  Iteration {iteration + 1}: ALG1 = {alg1_time:.2f}ms, ALG2 = {alg2_time:.2f}ms")

        alg1_avg = np.mean(alg1_iteration_times)
        alg2_avg = np.mean(alg2_iteration_times)

        results_remaining['n_values'].append(n)
        results_remaining['alg1_avg_times'].append(alg1_avg)
        results_remaining['alg2_avg_times'].append(alg2_avg)

        print(f"  Averages: ALG1 = {alg1_avg:.2f}ms, ALG2 = {alg2_avg:.2f}ms")
        print()

    return results_remaining

def main_continued():
    print("=== Continuing Closest Pair of Points Project ===\n")

    try:
        previous_results = np.load('project_results_partial_backup.npz')
        n_values_prev = previous_results['n_values'].tolist()
        alg1_avg_times_prev = previous_results['alg1_avg_times'].tolist()
        alg2_avg_times_prev = previous_results['alg2_avg_times'].tolist()
        print("Loaded previous results.")
    except FileNotFoundError:
        print("No previous partial results found. Starting fresh experiments.")
        n_values_prev = []
        alg1_avg_times_prev = []
        alg2_avg_times_prev = []


    print("\n Running remaining experiments...")
    results_remaining = run_experiments_again()

    # Combine results
    n_values_combined = n_values_prev + results_remaining['n_values']
    alg1_avg_times_combined = alg1_avg_times_prev + results_remaining['alg1_avg_times']
    alg2_avg_times_combined = alg2_avg_times_prev + results_remaining['alg2_avg_times']

    # Sort combined results by n_values
    sorted_indices = np.argsort(n_values_combined)
    n_values_combined_sorted = np.array(n_values_combined)[sorted_indices].tolist()
    alg1_avg_times_combined_sorted = np.array(alg1_avg_times_combined)[sorted_indices].tolist()
    alg2_avg_times_combined_sorted = np.array(alg2_avg_times_combined)[sorted_indices].tolist()

    combined_results = {
        'n_values': n_values_combined_sorted,
        'alg1_avg_times': alg1_avg_times_combined_sorted,
        'alg2_avg_times': alg2_avg_times_combined_sorted
    }

    print("\n Calculating constants and creating tables with combined results...")
    c1, c2, ratios_alg1, ratios_alg2 = calc_cnst_and_tables(combined_results)

    pred_alg1, pred_alg2 = gen_predictions(combined_results['n_values'], c1, c2)

    print("\n Generating plots with combined results...")
    create_plots(combined_results, pred_alg1, pred_alg2)

    print("\n Saving final results...")
    np.savez('project_results.npz',
             n_values=combined_results['n_values'],
             alg1_avg_times=combined_results['alg1_avg_times'],
             alg2_avg_times=combined_results['alg2_avg_times'],
             c1=c1, c2=c2)

if __name__ == "__main__":
    main_continued()

=== Continuing Closest Pair of Points Project ===

Loaded previous results.

 Running remaining experiments...
Testing n = 50000
  Iteration 1: ALG1 = 662058.57ms, ALG2 = 532.34ms
  Iteration 2: ALG1 = 663246.48ms, ALG2 = 848.36ms
  Iteration 3: ALG1 = 708182.02ms, ALG2 = 533.20ms
  Iteration 4: ALG1 = 702982.09ms, ALG2 = 1021.45ms
  Iteration 5: ALG1 = 714478.73ms, ALG2 = 569.95ms
  Iteration 6: ALG1 = 684014.81ms, ALG2 = 547.26ms
  Iteration 7: ALG1 = 683054.63ms, ALG2 = 910.49ms
  Iteration 8: ALG1 = 699618.16ms, ALG2 = 515.01ms
  Iteration 9: ALG1 = 705499.66ms, ALG2 = 737.63ms
  Iteration 10: ALG1 = 667605.10ms, ALG2 = 509.92ms
  Averages: ALG1 = 689074.02ms, ALG2 = 672.56ms

Testing n = 55000
  Iteration 1: ALG1 = 853060.35ms, ALG2 = 726.58ms
  Iteration 2: ALG1 = 847398.68ms, ALG2 = 643.21ms
  Iteration 3: ALG1 = 900062.70ms, ALG2 = 610.02ms
  Iteration 4: ALG1 = 854365.20ms, ALG2 = 692.46ms
  Iteration 5: ALG1 = 849286.90ms, ALG2 = 971.62ms
  Iteration 6: ALG1 = 859708.46ms, AL